In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
from getpass import getpass
from urllib.parse import quote

import pandas as pd
from SPARQLWrapper import JSON, SPARQLWrapper
from tqdm.auto import tqdm

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import s3, iiif

try:
    S3_CLIENT_ID
except NameError:
    S3_CLIENT_ID = getpass("Please input s3 client id")

try:
    S3_CLIENT_SECRET
except NameError:
    S3_CLIENT_SECRET = getpass("Please input s3 client secret")


tqdm.pandas()
def identifier_to_manifest_base_url(identifier):
    return "https://media.performing-arts.ch/iiif/manifest/" + quote(identifier)

def identifier_to_iiif_base_url(identifier):
    return "https://media.performing-arts.ch/iiif/3/image%2F" + identifier

In [2]:
identifier = "1602-1"
name = "Stiftung SAPA (CH-001439-9), Bestand Adolphe François Appia (A-1602), Bühnenbildentwürfe und -Skizzen (Serie)"
creator = "Adolphe François Appia"
image_identifiers = [f"A-1602-IK-{idx:03d}-IC" for idx in range(1,159)]
image_identifiers.remove("A-1602-IK-039-IC") # does not exists
image_identifiers.remove("A-1602-IK-143-IC") # does not exists
iiif_base_urls = [identifier_to_iiif_base_url(id) for id in image_identifiers]

In [7]:
manifest_base_url = identifier_to_manifest_base_url(identifier)
manifest_base_metadata = iiif.get_manifest_base_metadata(copyright="Public domain", rights="https://creativecommons.org/publicdomain/mark/1.0/")
manifest = iiif.create_manifest_from_iiif_images(
    manifest_base_url,
    [iiif.IIIFImageItem(iiif_base_url, label=iiif_base_url.split("image%2F")[1].replace("IK-", "IK/"), show_rendering=True) for iiif_base_url in iiif_base_urls],
    manifest_base_metadata,
    label=name,
    summary=name,
    sapa_resource="http://data.performing-arts.ch/r/736ee518-939d-4e25-809c-8b266c61b14e",
    sapa_resource_label = "Description of the Record Set on Swiss Performing Arts Platform",
    identifier=identifier,
    creator=creator
)

In [8]:
len(manifest['items'])

156

In [9]:
client = s3.S3Client(S3_CLIENT_ID, S3_CLIENT_SECRET)

client.upload_as_json(manifest, "manifests/" + identifier + ".json")

In [10]:
manifest

{'@context': 'http://iiif.io/api/presentation/3/context.json',
 'type': 'Manifest',
 'rights': 'https://creativecommons.org/publicdomain/mark/1.0/',
 'homepage': [{'id': 'https://sapa.swiss',
   'type': 'Text',
   'label': {'en': ['SAPA Homepage']},
   'format': 'text/html'}],
 'requiredStatement': {'label': {'en': ['Copyright'],
   'de': ['Copyright'],
   'fr': ['Copyright'],
   'it': ['Copyright']},
  'value': {'en': ['Public domain'],
   'de': ['Public domain'],
   'fr': ['Public domain'],
   'it': ['Public domain']}},
 'provider': [{'id': 'https://www.wikidata.org/entity/Q50920401',
   'type': 'Agent',
   'label': {'en': ['SAPA, Swiss Archive of the Performing Arts'],
    'de': ['Stiftung SAPA, Schweizer Archiv der Darstellenden Künste'],
    'fr': ['Fondation SAPA, Archives suisses des arts de la scène'],
    'it': ['Fondazione SAPA, Archivio svizzero delle arti della scena']},
   'homepage': [{'id': 'https://sapa.swiss/',
     'type': 'Text',
     'label': {'en': ['The SAPA Found